## Assembled Reformat

Example of creating product catalog pages.

In [1]:
GEMINI="gemini-2.0-flash"
#OPENAI="gpt-4o-mini"
#CLAUDE="claude-3-7-sonnet-latest"

import os
from dotenv import load_dotenv
load_dotenv("../keys.env")
assert os.environ["GEMINI_API_KEY"][:2] == "AI",\
       "Please specify the GEMINI_API_KEY access token in keys.env file"
#assert os.environ["ANTHROPIC_API_KEY"][:2] == "sk",\
#       "Please specify the ANTHROPIC_API_KEY access token in keys.env file"
#assert os.environ["OPENAI_API_KEY"][:2] == "sk",\
#       "Please specify the OPENAI_API_KEY access token in keys.env file"

In [2]:
# Needed in Jupyter environment See: https://ai.pydantic.dev/troubleshooting/ 
import nest_asyncio
nest_asyncio.apply()

## Gather snippets of information from database

For the items

In [7]:
from dataclasses import dataclass
from pydantic import Field

@dataclass
class CatalogContent:
    part_name: str = Field("Common name of part")
    part_id: str = Field("unique part id in catalog")
    part_description: str = Field("One paragraph description of part, which is one of the parts of a paper machine, explaining its purpose.")
    failure_modes: list[str] = Field("list of common reasons why customer might need to replace this part.")
    warranty_period: int = Field("number of years that the part is under warranty")
    price: str = Field("price of part")

def add_description(item: CatalogContent) -> CatalogContent:
    from pydantic_ai import Agent
    agent = Agent(GEMINI,
                  system_prompt="""
                  You are a content writer for a manufacturer of paper machines.
                  """,
                  model_settings = {
                      "temperature": 0.1   # not too creative??
                  },
                 result_type=CatalogContent)
    prompt=f"""
    Write content that will go in the Replacement Parts part of the manufacturer's website.
    I will give you an incomplete catalog content. Fill in the remaining fields.
    
    **Input**
    {item}
    """
    result = agent.run_sync(prompt)
    return result.data

item = add_description(CatalogContent("wet_end", "X34521PL", None, None, 3, "$23295"))
print(item)

CatalogContent(part_name='wet_end', part_id='X34521PL', part_description='The wet end of a paper machine is the section where the paper web is formed. It is arguably the most important section of the machine.', failure_modes=['Web breaks', 'Uneven sheet formation', 'Poor drainage'], warranty_period=3, price='$23295')


## Generate content

In [8]:
from pydantic_ai import Agent
agent = Agent(GEMINI,
              system_prompt="""You are a content marketer for a manufacturer of paper machines.""",
              model_settings = {
                      "temperature": 0.6   # creative
              })
prompt=f"""
Write content in Markdown that will go in the Replacement Parts part of the manufacturer's website.
Include a placeholder for an images, and include a description of the image.
Optimize the content for SEO. Also make it appealing to potential buyers.

**Part Information:**
{item}
"""
result = agent.run_sync(prompt)
print(result.data)

```markdown
# Maximize Uptime and Paper Quality with Genuine Wet End Replacement Parts

Is your paper machine's wet end performing optimally? The wet end is the heart of your paper production, and its performance directly impacts sheet formation, drainage, and ultimately, your profitability.  Don't let worn or damaged components compromise your paper quality or lead to costly downtime.  We offer a comprehensive range of genuine replacement parts for your paper machine's wet end, engineered to the highest standards for performance and longevity.

## The Importance of a High-Performing Wet End

The wet end is where the magic happens – the initial formation of the paper web.  A poorly functioning wet end can lead to a cascade of problems, including:

*   **Web Breaks:** Frequent web breaks are a major source of downtime and wasted materials.
*   **Uneven Sheet Formation:** Inconsistent sheet formation results in lower quality paper and potential customer complaints.
*   **Poor Drainage:**